In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from functools import partial
import os
import sys
import umap
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)

In [ ]:
# from synbio_morpher.utils.common.setup import construct_circuit_from_cfg, prepare_config

In [ ]:
sdir = '../'